<a href="https://colab.research.google.com/github/profmateusmodesto/Mobile_SSD_Tomato/blob/main/mobile_ssd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MONTANDO O GOOGLE DRIVE**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**INSTALANDO AS DEPENDÊNCIAS PARA O FUNCIONAMENTO DA ARQUITETURA**

In [ ]:
!pip install -U --pre tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pathlib
os.chdir('/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto')
print(os.getcwd())

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models
     

/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto


In [3]:
# Object detection API
%%bash
cd /content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#run model builder test
!python /content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/models/research/object_detection/builders/model_builder_tf2_test.py
     

2023-04-10 18:47:40.066677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 18:47:41.647735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
Running tests under Python 3.9.16: /usr/bin/pyth

**PREPARANDO A BASE DE DADOS DE TREINO**

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    classes_names = []

    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    classes_names = list(set(classes_names))
    classes_names.sort()

    return xml_df, classes_names

path_bd_train = '/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/train'

def main():
    for directory in ['train', 'test']:
        print(directory)
        image_path = os.path.join(os.getcwd(), path_bd_train.format(directory))
        xml_df, classes = xml_to_csv(image_path)
        print(os.getcwd())
        xml_df.to_csv( '/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/' + 'train_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')
    

main()

train
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto
Successfully converted xml to csv.
test
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto
Successfully converted xml to csv.


**PREPARANDO A BASE DE DADOS DE VALIDAÇÃO**

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    classes_names = []

    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    classes_names = list(set(classes_names))
    classes_names.sort()

    return xml_df, classes_names

path_bd_train = '/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/test'

def main():
    for directory in ['train', 'test']:
        print(directory)
        image_path = os.path.join(os.getcwd(), path_bd_train.format(directory))
        xml_df, classes = xml_to_csv(image_path)
        print(os.getcwd())
        xml_df.to_csv( '/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/' + 'valid_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')
    

main()

train
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto
Successfully converted xml to csv.
test
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto
Successfully converted xml to csv.


**CRIANDO TFRECORD**

Classes em números:

Ripe_tomatoes	= 1
Unripe_Tomatoes	= 2
Diseased = 3

In [5]:
# Convert csv to tf record files - To switch to a code based version so that the sys.exit dont take place.

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

"""
Usage:
  # From tensorflow/models/

  # Create train data:
  python Generate_TF_Records.py --csv_input=data/train_labels.csv  --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record

"""


#flags = tf.compat.v1.flags
#flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
#flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
#flags.DEFINE_string('image_dir', '', 'Path to images')
#LAGS = flags.FLAGS

# TO-DO replace this with label map

path_bd_train = '/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/test'

def class_text_to_int(row_label):
    if row_label == 'tree':
        return 1
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    print(path)
    print(group.filename)
    with tf.compat.v1.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
#'image/object/class/label': dataset_util.int64_list_feature(classes)
    return tf_example

def main(_):
    for directory in ['train', 'test']:
      output_path = os.path.join(os.getcwd(), '/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/train_bd.tfrecord'.format(directory))
      print(output_path)
      input_path = os.path.join(os.getcwd(), '/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/train_valid_labels.csv'.format(directory))
      print(input_path)
      writer = tf.io.TFRecordWriter(output_path)
      print(os.getcwd())
      path = os.path.join(os.getcwd(), path_bd_train.format(directory))
      print(path)
      examples = pd.read_csv(input_path)
      grouped = split(examples, 'filename')
      for group in grouped:
          tf_example = create_tf_example(group, path)
          writer.write(tf_example.SerializeToString())

      writer.close()
      #output_path = os.path.join(os.getcwd(), FLAGS.output_path)
      # print('Successfully created the TFRecords: {}'.format(output_path))

if __name__ == '__main__':
    tf.compat.v1.app.run()

/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/train_bd.tfrecord
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/train_valid_labels.csv
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/test
/content/gdrive/MyDrive/Pecege/Orientacao_de_TCC/Turma_211/Alunos/Igor_Barreto/Base_de_dados_Mateus/HARVESTING_VOC/Pixellib_padrao/test
-as-cooking-ingredients-halved-tomato-foreground-copyspace-top-view-59591442_jpg.rf.c169cbb74c0f3f79daa2443c9650fd4b.jpg


AttributeError: ignored